<a href="https://colab.research.google.com/github/waseGithub/saniWASE/blob/master/saniwase_output_Table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data output**

*This aims to provide top level stats on saniWASE under a 1 day index*

This script is linked to access data from 'saniWASE_backlog_cleaned'.
The data is printed to a dataframe at the bottom of the script. 
Current values: 
*   datetime (index) - day in reference 
*   tank_ID - id of the tank
*   ID - id of log (ignore this value)
*   CH4 - methane (%)
*   CO2 - Carbon dioxide (%)
*   OH - Other hydrocarbon (%)
*   Flow_1G_L - flow of tank for 1 day (L)




In [ ]:
#@title
# %%capture
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import numpy as np
from pathlib import Path
from google.colab import data_table
pd.set_option('display.max_rows', 1000)


def flow_timeinterval(data, time):
  data.reset_index(inplace =True)
  tanks = [1,2,3,4]
  ls = []
  for tank in tanks:
    data_tank = data.loc[data['ID'] == tank]
    data_tank['Flow_1H_L'] = data_tank['Flow_L'].shift(time) - data_tank['Flow_L']
    data_tank['Flow_1H_L'] = abs(data_tank['Flow_1H_L'].clip(upper=0))
    data_tank['Flow_1H_L'] = np.where((data_tank['Flow_L'] < data_tank['Flow_L'].shift(time)),data_tank['Flow_L'],data_tank['Flow_1H_L'])  
    ls.append(data_tank)

  data = pd.concat(ls, axis=0)
  data = data.set_index(['ID', 'datetime'])
  return data


def time_grouping(dframe, time):
  try:
    # dframe =  dframe.loc[(dframe['CO2'] >= 0) & (dframe['CO2'] <= 100)]
    dframe =  dframe.loc[(dframe['CH4'] >= 0) & (dframe['CH4'] <= 100)]
    dframe =  dframe.loc[(dframe['OH'] >= 0) & (dframe['OH'] <= 100)]
  except KeyError:
    print("key error")
    pass 
  df_csv_gas = dframe.groupby([pd.Grouper(freq=time, level='datetime'), pd.Grouper(level='ID')])['CH4', 'CO2', 'OH'].mean()
  df_csv_flow = dframe.groupby([pd.Grouper(freq=time, level='datetime'),pd.Grouper(level='ID')])['Flow_L'].max()
  df_csv_flow1H = dframe.groupby([pd.Grouper(freq=time, level='datetime'), pd.Grouper(level='ID')])['Flow_1H_L'].sum()
  df = pd.concat([df_csv_gas, df_csv_flow, df_csv_flow1H], axis=1)
  return df

Mounted at /content/gdrive


In [ ]:
#@title
%%capture
path2csv = Path("/content/gdrive/MyDrive/saniWASE_backlog_cleaned")
csvlist = path2csv.glob("*.csv")
ls = []
for g in csvlist:
  df_csv = pd.read_csv(g)
  df_csv['datetime'] = pd.to_datetime(df_csv['datetime'])
  df_csv = df_csv.set_index(['datetime', 'ID'])
  df_csv['Cnt'] = pd.to_numeric(df_csv.Cnt, errors='coerce')
  df_csv_gas = df_csv.groupby([pd.Grouper(freq='H', level='datetime'), 
                             pd.Grouper(level='ID')])['CH4', 'CO2', 'OH'].mean()
  df_csv_flow = df_csv.groupby([pd.Grouper(freq='H', level='datetime'), 
                             pd.Grouper(level='ID')])['Cnt'].max()
  df = pd.concat([df_csv_gas, df_csv_flow], axis=0)
            
  ls.append(df_csv)
df = pd.concat(ls, axis=0)
df['count'] = pd.to_numeric(df.Cnt, errors='coerce')
df['Flow_L'] = df['count'] * 10
df.reset_index(inplace =True)
df = df.sort_values(by=['datetime', 'ID'])
df.set_index(['datetime', 'ID'], inplace = True)



# Output table set period
Hour of day to calculate delta gass flow between (i.e 8 meaning 8am -8am) 

In [ ]:
Hour = 8 

In [ ]:
#@title
# %%capture
df2 = df 
df2.reset_index(inplace = True)
df2['hour'] = df2['datetime'].dt.time
df2 = df2.sort_values(by=['datetime', 'ID'])
df2['hour'] = df2['hour'].astype('str')
df2 = df2.loc[(df2['hour'] == '0' + str(Hour) + ':00:00') | (df2['hour'] == '0' + str(Hour) + ':05:00') | (df2['hour'] == '0' + str(Hour) + ':10:00')]
df2.set_index(['datetime', 'ID'], inplace = True)
df2.reset_index(inplace = True)



df2 = flow_timeinterval(df2, 1)#


df2 = time_grouping(df2, '1H')

df2.reset_index(inplace = True)
df2['tank_ID'] = df2['ID']
df2['tank_ID'] = df2['tank_ID'].replace([1,2,3,4], [2,3,1,4])
df2 = df2.sort_values(by=['datetime', 'tank_ID'])
df2.set_index(['datetime', 'tank_ID'], inplace =True)
df2.rename(columns = {'Flow_1H_L':'Flow_1D_L'}, inplace = True)
df2 = df2.drop(['Flow_L'], axis=1)
# display(df2)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
#@title
df2 = df2.round(2)
df2

ID    CH4     CO2    OH  Flow_1D_L
datetime            tank_ID                                    
2022-04-03 08:00:00 3         2   7.03    0.10  1.63        0.0
2022-06-05 08:00:00 1         3  56.60   15.20  3.83        0.0
                    2         1  52.20    4.70  3.83       10.0
                    3         2  67.77   13.50  3.87       30.0
                    4         4  74.90   10.40  3.77        0.0
2022-06-07 08:00:00 1         3  58.13   14.87  3.80     2180.0
                    2         1  50.70    4.97  3.90      590.0
                    3         2  65.57   13.80  3.83       70.0
                    4         4  72.63   11.03  3.73        0.0
2022-06-11 08:00:00 1         3  55.98   17.65  3.68      840.0
                    2         1  48.85    6.45  3.68      380.0
                    3         2  68.68   12.80  4.00       20.0
                    4         4  70.97   11.95  3.68        0.0
2022-06-12 08:00:00 1         3  55.60   18.13  3.13     1120.0
                    2         1  48.60    6.87  3.57      550.0
                    3         2  69.23   12.57  4.00       30.0
                    4         4  71.20   12.03  3.57        0.0
2022-06-13 08:00:00 1         3  56.38   18.38  4.00     1090.0
                    2         1  49.28    6.90  4.00      560.0
                    3         2  69.62   12.40  3.68       40.0
                    4         4  71.08   12.50  3.68        0.0
2022-06-14 08:00:00 1         3  53.88   19.52  4.00     1140.0
                    2         1  48.90    7.20  3.68      620.0
                    3         2  68.97   12.30  3.00       40.0
                    4         4  69.68   13.20  4.00        0.0
2022-06-15 08:00:00 1         3  52.70   20.40  4.00     1220.0
                    2         1  48.27    7.60  3.57      770.0
                    3         2  67.43   12.90  3.57       70.0
                    4         4  67.63   12.47  4.00        0.0
2022-06-16 08:00:00 1         3  55.47   19.57  3.57     1200.0
                    2         1  49.73    7.80  3.10      820.0
                    3         2  66.37   13.70  4.00       90.0
                    4         4  66.63   12.23  4.00        0.0
2022-06-17 08:00:00 1         3  57.00   19.53  4.00     1150.0
                    2         1  51.10    8.00  4.00      820.0
                    3         2  66.87   13.47  3.57       90.0
                    4         4  68.33   12.00  3.57        0.0
2022-06-18 08:00:00 1         3  56.20   19.70  4.00     1160.0
                    2         1  51.10    8.70  4.00      830.0
                    3         2  67.17   12.70  3.57      100.0
                    4         4  67.10   11.20  4.00        0.0
2022-06-19 08:00:00 1         3  56.97   19.13  4.00     1060.0
                    2         1  51.67    9.10  3.57      810.0
                    3         2  68.13   12.13  3.57      100.0
                    4         4  66.67   11.60  4.00        0.0
2022-06-20 08:00:00 1         3  63.43   16.47  4.00      860.0
                    2         1  54.40    8.80  3.57      710.0
                    3         2  69.93   12.00  3.57       80.0
                    4         4  64.93   12.30  3.57        0.0
2022-06-21 08:00:00 1         3  59.80   18.87  4.00      750.0
                    2         1  53.70    9.47  3.13      450.0
                    3         2  67.53   12.13  3.57       50.0
                    4         4  68.63   10.60  4.00        0.0
2022-06-22 08:00:00 1         3  45.17   26.97  4.00     1230.0
                    2         1  51.73   11.20  4.00      720.0
                    3         2  67.00   12.70  4.00      200.0
                    4         4  67.77   12.70  4.00        0.0
2022-06-29 08:00:00 1         3  78.17   19.50  4.00      820.0
                    2         1  49.05   15.30  4.00      560.0
                    3         2  70.93   11.70  4.00      300.0
                    4         4  71.47   11.00  4.00        0

In [ ]:
#@title
%%capture
data_table.enable_dataframe_formatter()
##
df = flow_timeinterval(df, 1)
# df


In [ ]:
#@title
%%capture
df = time_grouping(df, '1D')
##
df.rename(columns = {'Flow_1H_L':'Flow_1D_L'}, inplace = True)
df = df.drop(['Flow_L'], axis=1)
df.reset_index(inplace = True)
df['tank_ID'] = df['ID']
df['tank_ID'] = df['tank_ID'].replace([1,2,3,4], [2,3,1,4])
df = df.sort_values(by=['datetime', 'tank_ID'])
df.set_index(['datetime', 'tank_ID'], inplace =True)

# Ouput table

In [ ]:
#@title
df = df.round(2)
display(df)

ID    CH4         CO2    OH  Flow_1D_L
datetime   tank_ID                                        
2022-04-03 3         2   7.30        0.08  1.69        0.0
           4         4  72.10        6.90  3.30        0.0
2022-04-11 1         3   0.20        0.00  0.10        0.0
           2         1  40.20        0.30  3.80        0.0
           3         2  23.70        0.10  3.80       10.0
           4         4  39.90        5.50  3.10       10.0
2022-04-13 1         3  58.33       27.33  3.83        0.0
           3         2  75.57       15.73  3.77        0.0
           4         4  72.97        9.63  3.33        0.0
2022-06-03 1         3  53.21       16.49  3.81      350.0
           2         1  49.99        5.50  3.79       80.0
           3         2  68.59       12.85  3.78        0.0
           4         4  75.80        9.59  3.82        0.0
2022-06-04 1         3  58.43       14.33  3.82     1000.0
           2         1  50.66        5.26  3.71      550.0
           3         2  67.42       13.18  3.79       10.0
           4         4  74.15       10.45  3.80        0.0
2022-06-05 1         3  56.71       15.34  3.81     1090.0
           2         1  51.96        4.74  3.83      310.0
           3         2  67.70       13.50  3.81       40.0
           4         4  74.72       10.48  3.78        0.0
2022-06-06 1         3  57.71       15.22  3.81     1130.0
           2         1  50.93        4.92  3.81      320.0
           3         2  66.45       13.65  3.83       30.0
           4         4  73.64       10.61  3.83        0.0
2022-06-07 1         3  58.32       14.71  3.79     1050.0
           2         1  50.57        4.96  3.76      440.0
           3         2  65.58       13.78  3.81       40.0
           4         4  72.65       10.93  3.82        0.0
2022-06-08 1         3  60.74       13.39  3.85      560.0
           2         1  50.80        4.79  3.76      300.0
           3         2  64.16       13.07  3.80       20.0
           4         4  70.79       10.78  3.76        0.0
2022-06-09 1         3  61.17       13.96  3.80      920.0
           2         1  51.47        5.02  3.76      390.0
           3         2  65.67       13.14  3.77       30.0
           4         4  70.72       11.13  3.72        0.0
2022-06-10 1         3  57.54       16.42  3.83      600.0
           2         1  49.32        6.04  3.83      270.0
           3         2  66.93       12.92  3.80       40.0
           4         4  70.63       12.19  3.72        0.0
2022-06-11 1         3  55.74       17.82  3.76     1110.0
           2         1  48.62        6.58  3.82      530.0
           3         2  68.02       12.52  3.78       30.0
           4         4  70.32       12.19  3.80        0.0
2022-06-12 1         3  55.70       18.17  3.75     1100.0
           2         1  48.56        6.94  3.76      560.0
           3         2  68.77       12.32  3.82       30.0
           4         4  70.31       12.44  3.78        0.0
2022-06-13 1         3  55.57       18.73  3.78     1110.0
           2         1  49.26        6.97  3.80      570.0
           3         2  69.12       12.21  3.79       40.0
           4         4  70.09       12.69  3.79        0.0
2022-06-14 1         3  53.73       19.62  3.75     1200.0
           2         1  48.69        7.29  3.77      740.0
           3         2  68.67       12.18  3.68       60.0
           4         4  69.08       13.13  3.76        0.0
2022-06-15 1         3  52.92       20.26  3.80     1230.0
           2         1  48.27        7.75  3.79      820.0
           3         2  67.06       12.96  3.78       80.0
           4         4  67.57       13.06  3.81        0.0
2022-06-16 1         3  55.51       19.62  3.67     1160.0
           2         1  49.82        7.88  3.73      820.0
           3         2  66.50       13.63  3.85       90.0
           4         4  68.15       12.70  3.76        0.0
2022-06-17 1         3  56.42       19.38  3.78     1150.0
        

# Tank volume output table

In [ ]:
# @title
# %%capture
path2csv = Path("/content/gdrive/MyDrive/saniWASE_backlog_cleaned2")
csvlist = path2csv.glob("*.csv")
ls_tank = []
for g in csvlist:
  df_csv = pd.read_csv(g)
  df_csv['datetime'] = pd.to_datetime(df_csv['datetime'])
  df_csv = df_csv.set_index(['datetime'])
  # df_csv['Cnt'] = pd.to_numeric(df_csv.Cnt, errors='coerce')
  # df_csv_gas = df_csv.groupby([pd.Grouper(freq='H', level='datetime'), 
  #                            pd.Grouper(level='ID')])['CH4', 'CO2', 'OH'].mean()
  # df_csv_flow = df_csv.groupby([pd.Grouper(freq='H', level='datetime'), 
  #                            pd.Grouper(level='ID')])['Cnt'].max()
  # df = pd.concat([df_csv_gas, df_csv_flow], axis=0)
            
  ls_tank.append(df_csv)
df_tank = pd.concat(ls_tank, axis=0)
df_tank = df_tank.drop(['date'], axis=1)
df_tank.reset_index(inplace =True)
df_tank = df_tank.resample('30T', on='datetime').mean()
df_tank.reset_index(inplace =True)
df_tank = df_tank.sort_values(by=['datetime'])
df_tank.set_index(['datetime'], inplace = True)
df_tank.dropna(inplace=True)
df_tank = df_tank.round(1)
display(df_tank)

,Sensor_value,EQ_waste_height_mm,EQ_volume_%
datetime,,,
2022-06-29 17:30:00,880.6,541.8,80.0
2022-06-29 18:00:00,875.0,537.0,79.0
2022-06-29 23:00:00,778.8,454.8,70.7
2022-06-29 23:30:00,782.2,465.8,69.2
2022-06-30 09:30:00,618.0,338.0,50.0
2022-06-30 10:00:00,615.8,336.4,49.2
2022-06-30 10:30:00,608.1,330.8,48.3
2022-06-30 11:00:00,685.8,391.0,56.8
2022-06-30 11:30:00,928.2,578.6,84.2
